In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from scipy import optimize, linalg
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve

# Root Finding Solvers for Vector Functions

$$\begin{cases}
f_1(x_1,x_2,\cdots,x_n) &= 0 \\
f_2(x_1,x_2,\cdots,x_n) &= 0 \\
&\vdots \\
f_n(x_1,x_2,\cdots,x_n) &= 0
\end{cases}$$



**General nonlinear solvers**

- `fsolve(func, x0[, args, fprime, …])`: Find the roots of a function.
- `broyden1(F, xin[, iter, alpha, …])`: using Broyden’s first Jacobian approximation.
- `broyden2(F, xin[, iter, alpha, …])`: using Broyden’s second Jacobian approximation.

**Large-scale nonlinear solvers**

- `newton_krylov(F, xin[, iter, rdiff, method, …])`: using Krylov approximation for inverse Jacobian.
- `anderson(F, xin[, iter, alpha, w0, M, …])`: using (extended) Anderson mixing.

**Simple iteration solvers**

- `excitingmixing(F, xin[, iter, alpha, …])`: using a tuned diagonal Jacobian approximation.
- `linearmixing(F, xin[, iter, alpha, verbose, …])`: using a scalar Jacobian approximation.
- `diagbroyden(F, xin[, iter, alpha, verbose, …])`: using diagonal Broyden Jacobian approximation.

**[Example]** 2계 비선형 연립방정식의 해를 구하시오. ($a = -3$, $b = -1$)
$$\begin{cases}
x^2 + y^2 + a &= 0 \\
xy + b &= 0
\end{cases}$$

In [0]:
def eqns(u, a, b): # parameters: a, b
    x, y = u
    f1 = x**2 + y**2 + a
    f2 = x*y + b
    return f1, f2

In [18]:
optimize.fsolve(eqns, [0.5,1.5], args=(-3,-1), full_output=True)

(array([0.61803399, 1.61803399]), {'fjac': array([[-0.58381252, -0.8118885 ],
         [ 0.8118885 , -0.58381252]]),
  'fvec': array([ 0.00000000e+00, -2.22044605e-16]),
  'nfev': 8,
  'qtf': array([-9.20708768e-10,  4.83706707e-10]),
  'r': array([-1.9231426 , -2.50318187,  2.32543004])}, 1, 'The solution converged.')

# Unified Root Finding Solver `scipy.optimize.root`

```python
optimize.root(fun, x0, args=(), method='hybr', jac=None, tol=None, 
            callback=None, options=None)
```

- `method='hybr', 'lm', 'broyden1', 'broyden2', 'anderson', 'linearmixing', 'diagbroyden', 'excitingmixing', 'krylov', 'df-sane'`

In [19]:
optimize.root(eqns, [0.5,1.5], args=(-3,-1), method='broyden1')

     fun: array([-7.39335442e-08, -6.59721966e-09])
 message: 'A solution was found at the specified tolerance.'
     nit: 9
  status: 1
 success: True
       x: array([0.61803399, 1.61803396])

In [20]:
optimize.root(eqns, [0.5,1.5], args=(-3,-1), method='broyden2')

     fun: array([-1.73857647e-06,  2.84360996e-07])
 message: 'A solution was found at the specified tolerance.'
     nit: 8
  status: 1
 success: True
       x: array([0.61803443, 1.61803328])

# Newton's Method

$n$개의 비선형 연립방정식에 대해,
$$\begin{cases}
f_1(x_1,x_2,\cdots,x_n) = &0 \\
f_2(x_1,x_2,\cdots,x_n) = &0 \\
&\vdots \\
f_n(x_1,x_2,\cdots,x_n) = &0
\end{cases}$$

임의의 한 점 $\mathbf{x}$에서 개별 함수 $f_i(\mathbf{x})$의 Taylor series는 다음과 같이 주어진다.
$$f_i(\mathbf{x}+\mathbf{\Delta x}) = f_i(\mathbf{x}) + \sum_{j=1}^n \frac{\partial f_i}{\partial x_j}\Delta x_j + O(\Delta x^2)$$

$O(\Delta x^2)$ 이상의 항을 무시하면, 다음과 같은 벡터식으로 나타낼 수 있다.
$$\mathbf{f}(\mathbf{x}+\mathbf{\Delta x}) = \mathbf{f}(\mathbf{x}) + \mathbf{J}(\mathbf{x})\,\mathbf{\Delta x}, \quad J_{ij} = \frac{\partial f_i}{\partial x_j}$$
여기서, $\mathbf{J}(\mathbf{x})$은 Jacobian matrix $(n\times n)$ 또는 Jacobian 이라고 부른다.

$\mathbf{x}$를 $f_i(\mathbf{x}) = \mathbf{0}$의 근으로 가정하면, $\mathbf{x}+\mathbf{\Delta x}$는 그 다음 근으로 볼 수 있다. 보정항인 $\mathbf{\Delta x}$를 구하기 위해 $\mathbf{f}(\mathbf{x}+\mathbf{\Delta x}) = \mathbf{0}$로 두면, 다음과 같은 $\mathbf{\Delta x}$에 대한 선형 연립방정식를 얻을 수 있다.
$$\mathbf{J}(\mathbf{x})\,\mathbf{\Delta x} = - \mathbf{f}(\mathbf{x}), \qquad\therefore \boxed{\mathbf{x}_{k+1} = \mathbf{x}_k - \mathbf{J}^{-1}\,\mathbf{f}(\mathbf{x})}$$

매번 각각의 Jacobian 항을 구하는 것은 어렵고 비효율적일 수 있다. 이 때는 유한 차분법을 이용하여 근사할 수 있다.
$$\frac{\partial f_i}{\partial x_j} \simeq \frac{f_i(\mathbf{x}+\hat{\mathbf{x}}_jh) - f_i(\mathbf{x})}{h}$$

또는 automatic differentiation을 이용하면 자동으로 jacobian을 계산할 수 있다. 여기에서는 pytorch 모듈을 이용하였다.

## Jacobian Using Automatic Differentiation

In [21]:
# Jacobian using pytorch
def jac_ad(func, u0, args):
    u = torch.tensor(u0, dtype=torch.float64, requires_grad=True)
    n = len(u0)
    f = func(u, *args)
    jac = torch.empty((n,n), dtype=torch.float64)
    for i in range(n):
        jac[i] = torch.autograd.grad(f[i], u)[0]
    return jac.numpy()

def newton_sys(func, x0, args=(), tol=1e-12, maxiter=100):
    x1 = np.asfarray(x0)
    for k in range(maxiter):
        f1 = func(x1, *args)
        jac = jac_ad(func, x1, args) # Automatic differentiation
        x2 = x1 - linalg.solve(jac, f1)
        f2 = func(x2, *args)
        if linalg.norm(x2-x1) < tol:
            break
        else:
            x1 = x2
    return x2, k+1

newton_sys(eqns, [0.5,1.5], args=(-3,-1))

(array([0.61803399, 1.61803399]), 5)

## Jacobian Using Numerical Differentiation

In [22]:
# Jacobian using FDM
def jac_num(f, x0, args, h=1E-6):
    x1 = x0.copy()
    f1 = np.asarray(f(x1, *args))
    jac = np.zeros((x0.size,x0.size))
    for i in range(x0.size):
        x1[i] += h
        jac[:,i] = (f(x1, *args)-f1)/h
    return jac

def newton_sys2(func, x0, args=(), tol=1e-12, maxiter=100):
    x1 = np.asfarray(x0)
    for k in range(maxiter):
        f1 = func(x1, *args)
        jac = jac_num(func, x1, args) # Numerical differentiation
        x2 = x1 - linalg.solve(jac, f1)
        f2 = func(x2, *args)
        if linalg.norm(x2-x1) < tol:
            break
        else:
            x1 = x2
    return x2, k+1

newton_sys2(eqns, [0.5,1.5], args=(-3,-1))

(array([0.61803399, 1.61803399]), 9)